In [1]:
from tensorflow import keras

In [2]:
def get_keras_dataset():
  (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
  
  # Add channel dimension so it works with keras.layers.Conv2D
  x_train = x_train.reshape(-1, 28, 28, 1)
  x_test = x_test.reshape(-1, 28, 28, 1)
  
  return (x_train, y_train), (x_test, y_test)


In [4]:
#!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 281.3 MB 1.0 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 198 kB 1.3 MB/s            
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805913 sha256=4ee0d6fc13561e75fd8d29327fe1c2c0f32528c5083cb8771476416196f8e6f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-g6pau14d/wheels/23/f6/d3/110e53bd43baeb8d7d38049733d48e39cbecd056f01dba7ee8
Successfully built pyspark


In [3]:
import random

from scipy import ndimage

from pyspark.sql.types import IntegerType, StructField, StructType
from pyspark.ml.image import ImageSchema


In [4]:
uint8_one_channel = ImageSchema.ocvTypes['CV_8UC1']

AttributeError: 'NoneType' object has no attribute '_jvm'

In [8]:
def spark_image(numpy_image):
  height, width, num_chan = numpy_image.shape
  data = bytearray(numpy_image.tobytes())
  return ['', height, width, num_chan, uint8_one_channel, data]

enhancment_factor = 5

def add_rotation(example):
  """Make rotated images from the orginal example"""
  image, label = example
  for i in range(enhancment_factor):
    angle = random.randint(-45, 45)
    rotatedImage = ndimage.rotate(image, angle, reshape=False).astype('uint8')
    yield [spark_image(rotatedImage), label]

schema = (StructType()
  .add(StructField("image", ImageSchema.columnSchema))
  .add(StructField("label", IntegerType())))

(x_train, y_train), (x_test, y_test) = get_keras_dataset()
enhanced_data = (sc.parallelize(zip(x_train, y_train.tolist()), 8)
  .flatMap(add_rotation)
  .toDF(schema))
  
display(enhanced_data)

AttributeError: 'NoneType' object has no attribute '_jvm'

In [6]:
!pip install sagemaker

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 481 kB 17.6 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 131 kB 56.4 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 81 kB 50.7 MB/s            
     |████████████████████████████████| 8.5 MB 1.3 MB/s             
     |████████████████████████████████| 79 kB 12.0 MB/s            
     |████████████████████████████████| 65 kB 16.3 MB/s            
     |████████████████████████████████| 128 kB 17.1 MB/s            
     |████████████████████████████████| 86 kB 22.5 MB/s            
     |████████████████████████████████| 138 kB 19.1 MB/s            
  Created wheel for sagemaker: filename=sagemaker-2.73.0-py2.py3-none-any.whl size=662723 sha256=fc32687272b22a7ca79e95b6aa09c97737207911898c8b4633412

In [7]:
import sagemaker

In [8]:
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

ValueError: Must setup local AWS configuration with a region supported by SageMaker.

In [9]:
!python generate_cifar10_tfrecords.py --data-dir ./data

python: can't open file 'generate_cifar10_tfrecords.py': [Errno 2] No such file or directory
